# Wrapping a fine-tuned Transformer

---
This tutorial will briefly take your through how to wrap an already trained transformer for text classification in a spacy pipeline. For this example we will use DaNLP's BertTone as an example. However do note that this approach also works using models directly from Huggingface's model hub.

Before we start let us make sure everything is installed:


In [ ]:
!pip -q install git+https://github.com/KennethEnevoldsen/DaCy

## Wrapping a Huggingface model
---
It is now easy to wrap a huggingface model it can be done simply in one line of code:


In [ ]:
from dacy.subclasses import add_huggingface_model
nlp = spacy.blank("da")  # replace with your desired pipeline
nlp = add_huggingface_model(nlp, 
                      download_name="pin/senda", # the model name on the huggingface hub
                      doc_extention="senda_trf_data", # the doc extention transformer data e.g. including wordpieces
                      model_name="senda",  # the name of the model in the pipeline
                      category="polarity", # the category type it predicts
                      labels=["negative", "neutral", "positive"], # possible outcome labels
                      )

## Step by step
---
However, one might want to look a bit more into it so let is go through step by step. First of all let is start with the setup. We will here use a downloaded huggingface transformer from DaNLP.

### Setup
Let's start of by downloading the model as be sure that it can be loaded in using Huggingface transformers:

In [ ]:
from danlp.download import download_model as danlp_download
from danlp.download import _unzip_process_func
from danlp.download import DEFAULT_CACHE_DIR as DANLP_DIR
from transformers import AutoModelForSequenceClassification, BertTokenizer

# downloading model and setting a path to its location
path_sub = danlp_download(
    "bert.polarity", DANLP_DIR, process_func=_unzip_process_func, verbose=True
)
path_sub = os.path.join(path_sub, "bert.pol.v0.0.1")

# loading it in with transformers
berttone = AutoModelForSequenceClassification.from_pretrained(path_sub, num_labels=3)

Assuming this works you are ready to move on. However if you were to do this on your own model I would also test that the forward pass works as intended.

## Wrapping the Model

---

Now we will start wrapping the model. DaCy provides good utility function for doing precisely this without make more changes than necessary to the transformer class from SpaCy. This should allow you to use the extensive documentation by SpaCy while working with this code.

This is utilizes SpaCy config system, which might take a bit of getting used to initially, but it is quite worth the effort. For now I will walk you through it:


In [6]:
from dacy.subclasses import ClassificationTransformer, install_classification_extensions

labels=["positive", "neutral", "negative"]
doc_extension = "berttone_pol_trf_data"
category = "polarity"

config = {
    "doc_extention_attribute": doc_extension,
    "model": {
        "@architectures": "dacy.ClassificationTransformerModel.v1",
        "name": path_sub,
        "num_labels": len(labels),
    },
}


# add the relevant extentsion to the doc
install_classification_extensions(
    category=category, labels=labels, doc_extention=doc_extension
)

The config files is an extention of the `Transformers` config in SpaCy, but you will note a few changes:

- 1) `doc_extention_attribute`: This is to make sure that the doc extension can be customized. The doc extentsion is how you fetch data relevant to your model. The SpaCy transformer uses the `trf_data`, but we don't want to overwrite this in case we are using multiple transformers.
- 2) `num_labels`: The number of labels. This is an argument passed forward when loading the model. Without this the Huggingface transformers package will raise an error (at least for cases where `num_labels` isn't 2)

Some might also ask regarding the `name`, but this simply specifies the name of the model. You could potentially change this out for any sequence classification model on Huggingfaces model hub. Lastly the `install_classification_extensions` adds the getter function for the model. Here it would for instance add `doc._.polarity` for extracting the label of the model as well as a `doc._.polarity_prop` for extracting the polarity probabilities for each class.


## Adding it to the NLP pipeline
---

Now it can simply add it to the pipeline using the `add_pipe`

In [9]:
import spacy
nlp = spacy.blank("da") # dummy nlp

clf_transformer = nlp.add_pipe(
    "classification_transformer", name="berttone", config=config
)
clf_transformer.model.initialize()


## Final Test

--- 

We can then finish off with a final test to see if everything works as intended:

In [10]:
texts = ["Analysen viser, at økonomien bliver forfærdelig dårlig", 
         "Jeg tror alligvel, det bliver godt"]

docs = nlp.pipe(texts)

for doc in docs:
    print(doc._.polarity)
    print(doc._.polarity_prop)

negative
{'prop': array([0.002, 0.008, 0.99 ], dtype=float32), 'labels': ['positive', 'neutral', 'negative']}
positive
{'prop': array([0.854, 0.146, 0.001], dtype=float32), 'labels': ['positive', 'neutral', 'negative']}


In [11]:
# we can also examine the wordpieces used using (and see the entire TransformersData)

doc._.berttone_pol_trf_data

TransformerData(wordpieces=WordpieceBatch(strings=[['[CLS]', 'jeg', 'tror', 'alli', '##g', '##vel', ',', 'det', 'bliver', 'godt', '[SEP]']], input_ids=array([[    2,   102,  1421,  9682, 31704,  1041,   883,    49,   352,
          380,     3]]), attention_mask=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), lengths=[11], token_type_ids=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])), tensors=[array([[ 3.3191876,  1.5510517, -4.0984917]], dtype=float32)], align=Ragged(data=array([[1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]], dtype=int32), lengths=array([1, 1, 3, 1, 1, 1, 1], dtype=int32), data_shape=(-1,), cumsums=None))